## Importing the necessary libraries

In [1]:
import os
import cv2
import numpy as np
import glob
import imutils
from keras.preprocessing.image import img_to_array,load_img
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose,Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping


KeyboardInterrupt



In [2]:
os.chdir('C:\\Users\\win10')
owd = os.getcwd()
print(owd)
os.chdir(owd)

C:\Users\win10


## Setting directory path variable and Function to process and store video frames

In [3]:
store_image=[]
train_path='C:\\Projects\\Video Surveilance\\Train path'
fps=5
train_videos=os.listdir('C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\training_videos')
print(train_videos)
train_images_path=train_path+'/frames'
# os.makedirs(train_images_path)

['01.mp4', '02.mp4', '03.mp4', '04.mp4', '05.mp4', '06.mp4', '07.mp4', '08.mp4', '09.mp4', '10.mp4', '11.mp4', '12.mp4', '13.mp4', '14.mp4', '15.mp4', '16.mp4']


## Extract frames from video and call store function

In [5]:
# print(train_videos)
os.chdir(train_images_path)
# for video in train_videos:
#     print(video)
vid = [cv2.VideoCapture(f'C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\training_videos\\{path}') for path in train_videos]
print(vid)
v=1
for vidcap in vid:
    success,image = vidcap.read()
    count=1;
    while success:
        cv2.imwrite("Vid"+str(v)+"-%d.jpg" % count, image)
        success,image = vidcap.read()
        count += 1
    print("Video -",count)
    v+=1

[<VideoCapture 0000026D75102C10>, <VideoCapture 0000026D69EA9CD0>, <VideoCapture 0000026D69EA9C30>, <VideoCapture 0000026D751D5210>, <VideoCapture 0000026D751D5050>, <VideoCapture 0000026D69EA9BB0>, <VideoCapture 0000026D751D51F0>, <VideoCapture 0000026D751D50D0>, <VideoCapture 0000026D751D53B0>, <VideoCapture 0000026D751D5230>, <VideoCapture 0000026D751D53D0>, <VideoCapture 0000026D751D52B0>, <VideoCapture 0000026D751D52D0>, <VideoCapture 0000026D751D5070>, <VideoCapture 0000026D751D51B0>, <VideoCapture 0000026D751D5290>]
Video - 1365
Video - 1512
Video - 1488
Video - 1512
Video - 816
Video - 1512
Video - 1100
Video - 1018
Video - 1392
Video - 1224
Video - 782
Video - 146
Video - 367
Video - 511
Video - 354
Video - 245


## Store the store_image list in a numpy file

In [4]:
os.chdir(train_path)
store_image = []

def store_inarray(image_path):
    image=load_img(image_path)
    image=img_to_array(image)
    image=cv2.resize(image, (227,227), interpolation = cv2.INTER_AREA)
    gray=0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)

images=os.listdir(train_images_path)
i=0
for image in images:
    i+=1
    if i<=6000:
        if(i%100==0):
            print(i,"Images Processed")
        
        image_path=train_images_path + '\\' + image
        store_inarray(image_path)    

print("All Images Processed")
store_image=np.array(store_image)
a,b,c = store_image.shape
print(a,b,c)

100 Images Processed
200 Images Processed
300 Images Processed
400 Images Processed
500 Images Processed
600 Images Processed
700 Images Processed
800 Images Processed
900 Images Processed
1000 Images Processed
1100 Images Processed
1200 Images Processed
1300 Images Processed
1400 Images Processed
1500 Images Processed
1600 Images Processed
1700 Images Processed
1800 Images Processed
1900 Images Processed
2000 Images Processed
2100 Images Processed
2200 Images Processed
2300 Images Processed
2400 Images Processed
2500 Images Processed
2600 Images Processed
2700 Images Processed
2800 Images Processed
2900 Images Processed
3000 Images Processed
3100 Images Processed
3200 Images Processed
3300 Images Processed
3400 Images Processed
3500 Images Processed
3600 Images Processed
3700 Images Processed
3800 Images Processed
3900 Images Processed
4000 Images Processed
4100 Images Processed
4200 Images Processed
4300 Images Processed
4400 Images Processed
4500 Images Processed
4600 Images Process

In [5]:
store_image.resize(b,c,a)
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1)
np.save('training.npy',store_image)

In [6]:
stae_model=Sequential()
stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))
stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [7]:
stae_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 55, 55, 10, 128)   15616     
                                                                 
 conv3d_1 (Conv3D)           (None, 26, 26, 10, 64)    204864    
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 26, 26, 10, 64)    295168    
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 26, 26, 10, 32)    110720    
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 26, 26, 10, 64)    221440    
                                                                 
 conv3d_transpose (Conv3DTra  (None, 55, 55, 10, 128)  204928    
 nspose)                                                         
                                                        

In [8]:
training_data=np.load('training.npy')
frames=training_data.shape[2]
frames=frames-frames%10
training_data=training_data[:,:,:frames]
training_data=training_data.reshape(-1,227,227,10)
training_data=np.expand_dims(training_data,axis=4)
target_data=training_data.copy()
epochs=3
batch_size=1
callback_save = ModelCheckpoint("saved_model.h5", monitor="mean_squared_error", save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)
stae_model.fit(training_data,target_data, batch_size=batch_size, epochs=epochs, callbacks = [callback_save,callback_early_stopping])
stae_model.save("saved_model.h5")

Epoch 1/3
600/600 [==============================] - 514s 836ms/step - loss: 0.0864 - accuracy: 0.7092
Epoch 2/3
600/600 [==============================] - 499s 832ms/step - loss: 0.0487 - accuracy: 0.7521
Epoch 3/3
600/600 [==============================] - 503s 839ms/step - loss: 0.0289 - accuracy: 0.7738


# TESTING

In [2]:
import cv2
import numpy as np 
# import tensorflow as tf
# import argparse
from PIL import Image
import imutils
from keras.models import load_model

ImportError: cannot import name 'KerasLazyLoader' from 'tensorflow.python.util.lazy_loader' (e:\Personal Projects\Video Surveilence System\venv\lib\site-packages\tensorflow\python\util\lazy_loader.py)

In [16]:
def mean_squared_loss(x1,x2):
    difference=x1-x2
    a,b,c,d,e=difference.shape
    n_samples=a*b*c*d*e
    sq_difference=difference**2
    Sum=sq_difference.sum()
    distance=np.sqrt(Sum)
    mean_distance=distance/n_samples
    return mean_distance

In [19]:
model=tf.keras.models.load_model("./saved_model.h5")
cap = cv2.VideoCapture("./Avenue_Dataset/testing_videos/03.avi")
print(cap.isOpened())

True


In [20]:
while cap.isOpened():
    imagedump=[]
    ret,frame=cap.read()
    for i in range(10):
        ret,frame=cap.read()
        image = imutils.resize(frame,width=700,height=600)
        frame=cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)
    imagedump=np.array(imagedump)
    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)
    output=model.predict(imagedump)
    loss=mean_squared_loss(imagedump,output)
    if frame.any()==None:
        print("none")
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if loss>0.00068:
        print('Abnormal Event Detected')
        cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
    cv2.imshow("video",image)
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import cv2
import numpy as np 
from keras.models import load_model
import argparse
from PIL import Image
import imutils


def mean_squared_loss(x1,x2):
    difference=x1-x2
    a,b,c,d,e=difference.shape
    n_samples=a*b*c*d*e
    sq_difference=difference**2
    Sum=sq_difference.sum()
    distance=np.sqrt(Sum)
    mean_distance=distance/n_samples

    return mean_distance


model=load_model("C:\\Projects\\Video Surveilance\\Train path\\saved_model.h5")

cap = cv2.VideoCapture("C:\\Projects\\Video Surveilance\\Avenue_Dataset\\Avenue Dataset\\testing_videos\\03.avi")
print(cap.isOpened())

while cap.isOpened():
    imagedump=[]
    ret,frame=cap.read()

    for i in range(10):
        ret,frame=cap.read()
        image = imutils.resize(frame,width=700,height=600)

        frame=cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)

    imagedump=np.array(imagedump)

    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)

    output=model.predict(imagedump)

    loss=mean_squared_loss(imagedump,output)

    if frame.any()==None:
        print("none")

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
    if loss>0.00068:
        print('Abnormal Event Detected')
        cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)

    cv2.imshow("video",image)

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np 
from keras.models import load_model
import argparse
from PIL import Image
import imutils

def mean_squared_loss(x1,x2):
    difference=x1-x2
    a,b,c,d,e=difference.shape
    n_samples=a*b*c*d*e
    sq_difference=difference**2
    Sum=sq_difference.sum()
    distance=np.sqrt(Sum)
    mean_distance=distance/n_samples

    return mean_distance

model=load_model("./saved_model.h5")

cap = cv2.VideoCapture("./Avenue_Dataset/testing_videos/05.avii")
print(cap.isOpened())

while cap.isOpened():
    imagedump=[]
    ret,frame=cap.read()

    for i in range(10):
        ret,frame=cap.read()
        image = imutils.resize(frame,width=700,height=600)
        frame=cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray=0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray=(gray-gray.mean())/gray.std()
        gray=np.clip(gray,0,1)
        imagedump.append(gray)

    imagedump=np.array(imagedump)

    imagedump.resize(227,227,10)
    imagedump=np.expand_dims(imagedump,axis=0)
    imagedump=np.expand_dims(imagedump,axis=4)

    output=model.predict(imagedump)

    loss=mean_squared_loss(imagedump,output)

    if frame.any()==None:
        print("none")

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
    print(loss)
    if loss>=0.00023:
        print('Abnormal Event Detected')
        cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)

    cv2.imshow("video",image)

cap.release()
cv2.destroyAllWindows()

True
0.00021849622041102845
0.00021924999777083183
0.00022024953617262417
0.0002183109716554327
0.0002191211117955691
0.00021972001287743332
0.00021961474001450562
0.00022113025393297753
0.00021920022802751835
0.00022015449153601246
0.0002209990893762133
0.00022215457337906828
0.00022158591095609642
0.0002204127025485939
0.0002202239893751617
0.00022210723688424972
0.00022314079838648099
0.00022277581323007752
0.00022230758690404046
0.00022295144893349716
0.00022575357019489358
0.0002270434058736443
0.00022809388909687526
0.00022956226607334586
0.00023142182828336905
0.00023355890788769474
0.00023502629969890108
0.0002337086401567939
0.00023761510980276643
0.00023706428339662512
0.00023864983046418991
0.00024217422051373163
0.0002513705400480156
Abnormal Event Detected
0.00024891787021720887
0.00025629840628754686
Abnormal Event Detected
0.0002619384687095425
Abnormal Event Detected
0.0002593721435120613
Abnormal Event Detected
0.0002570033302075653
Abnormal Event Detected
0.0002636079

AttributeError: 'NoneType' object has no attribute 'shape'